# Decision Tree Classifier

Hello, this is an implementation using Python 3 of a decision tree. This work was developed by Clarissa Lima for her studies on Machine Learning.

Mode Clarissa Codes: https://github.com/Clalloures

If you see any error, sorry, please contact me if possible :)

## Let's Start \o/

The first thing we need is to have a dataset. In this case we will use the idea of a set of foods to make the test

#### Toy example:

- Foods in my house
- Each row is an example in this dataset
- The last column is the label.
- The first three columns are features.

Let's create a header for this dataset, this will only appear when we start the table:


In [4]:
header = ["type", "color", "price in dolar", "label"]

In [5]:
training_data = [
    ['Fruit','Green', 3, 'Apple'],
    ['Fruit','Red', 3, 'Apple'],
    ['Fruit','Red', 1, 'Grape'],
    ['Fruit','Red', 1, 'Grape'],
    ['Fruit','Yellow', 3, 'Lemon'],
    ['Fruit','Orange',3, 'Orange'],
    ['Fruit','Yellow', 5, 'Pineapple'],
    ['Fruit','Yellow', 3, 'Mango'],
    ['Fruit','Purple', 1, 'Grape'],
    ['Fruit','Yellow', 5, 'Banana'],
    ['Drink','Brown' ,5 ,'Coffee'],
    ['Drink','Orange',4,'Orange Juice'],
    ['Drink','Red',2,'Grape Juice'],
    ['Drink','Yellow',5,'Lemonade'],
    ['Drink','Orange',4,'Orange Soda'],
    ['Drink','Brown', 5, 'Hot Chocolate'],
    ['Drink','Green', 2, 'Green Tea']
]

We have many values repeated, since "in my house" there can be more than one single object from the same type. We can use some functions to calculate how many unique values we have in a column in this dataset

In [6]:
def unique_values(rows, column):
    return set([row[column] for row in rows])

Let's see an example if we apply this function in column 0 and then an example if we apply the function in column 1

In [7]:
unique_values(training_data, 0)

{'Drink', 'Fruit'}

Note that the function returned in alphabetical order the values :)

In [9]:
unique_values(training_data, 1)

{'Brown', 'Green', 'Orange', 'Purple', 'Red', 'Yellow'}

Now, let's count the number of each type of example in this dataset

In [11]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [12]:
class_counts(training_data)

{'Apple': 2,
 'Grape': 3,
 'Lemon': 1,
 'Orange': 1,
 'Pineapple': 1,
 'Mango': 1,
 'Banana': 1,
 'Coffee': 1,
 'Orange Juice': 1,
 'Grape Juice': 1,
 'Lemonade': 1,
 'Orange Soda': 1,
 'Hot Chocolate': 1,
 'Green Tea': 1}

In [13]:
def class_counts2(rows):
    counts = {}
    for row in rows:
        label = row[0]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [14]:
class_counts2(training_data)

{'Fruit': 10, 'Drink': 7}

In your case we need to use the first function, because your label is aways in the last colum.

In [22]:
def is_value_numeric(value):
    if isinstance(value, int) == 1 or isinstance(value, float) == 1:
        return True
    else:
        return False

In [23]:
#######
# Demo:
print(is_value_numeric(1))
print(is_value_numeric("1"))
#######

True
False


Now, we need to create a question.
The question is used to partition a dataset.

We records a Column number and a Column value.
The match is used to compare the value in a example to the value stored in the Question

In [24]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

categorical attribute examples:

In [35]:
type_question = Question(0,'Drink')
f

Is type == Drink?

In [36]:
color_question = Question(1, 'Green')
q

Is color == Green?

numeric attribute example:

In [37]:
price_question = Question(2, 3)
price_question

Is price in dolar >= 3?

Now, we will pick an example from the training set aind will see if it matches with the question :)

In [38]:
example = training_data[0]
color_question.match(example)

True

In [39]:
type_question.match(example)

False

### Partitions a dataset. 
For each row in the dataset, we will check if it matches with the question. 
- Add it to 'true rows' 
- Or add it to 'false rows'

In [40]:
def partition(rows, question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

Example: 
    Partition the training data based on whether rows are Drinks.

In [42]:
true_rows, false_rows = partition(training_data, Question(0, 'Drink'))
true_rows

[['Drink', 'Brown', 5, 'Coffee'],
 ['Drink', 'Orange', 4, 'Orange Juice'],
 ['Drink', 'Red', 2, 'Grape Juice'],
 ['Drink', 'Yellow', 5, 'Lemonade'],
 ['Drink', 'Orange', 4, 'Orange Soda'],
 ['Drink', 'Brown', 5, 'Hot Chocolate'],
 ['Drink', 'Green', 2, 'Green Tea']]

In [43]:
false_rows

[['Fruit', 'Green', 3, 'Apple'],
 ['Fruit', 'Red', 3, 'Apple'],
 ['Fruit', 'Red', 1, 'Grape'],
 ['Fruit', 'Red', 1, 'Grape'],
 ['Fruit', 'Yellow', 3, 'Lemon'],
 ['Fruit', 'Orange', 3, 'Orange'],
 ['Fruit', 'Yellow', 5, 'Pineapple'],
 ['Fruit', 'Yellow', 3, 'Mango'],
 ['Fruit', 'Purple', 1, 'Grape'],
 ['Fruit', 'Yellow', 5, 'Banana']]

### Gini Impurity
We will now calculate the Gini Impurity for a list of rows. 
There are a few different ways to do this.

In [47]:
def gini_calculation(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

Example of a dataset without impurity values:

In [48]:
# First, we'll look at a dataset with no mixing.
no_mixing = [['Fruit', 'Yellow', 3, 'Mango'],
             ['Fruit', 'Yellow', 3, 'Mango'],
             ['Fruit', 'Yellow', 3, 'Mango']]
# this will return 0
gini_calculation(no_mixing)

0.0

In [50]:
middle_mixing = [['Fruit', 'Yellow', 3, 'Mango'],
                 ['Fruit', 'Yellow', 3, 'Lemon']]
gini_calculation(middle_mixing)

0.5

Our dataset:

In [51]:
gini_calculation(training_data)

0.9134948096885807

### Information Gain. 
Calculate the uncertainty of the starting node, minus the weighted impurity of two child nodes.

In [59]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

Example in our dataset. How much information do we gain by partioning on 'Drink'?

In [60]:
true_rows, false_rows = partition(training_data, Question(0, 'Drink'))
info_gain(true_rows, false_rows, gini(training_data))

0.07820069204152197

In [61]:
true_rows, false_rows = partition(training_data, Question(0,'Fruit'))
info_gain(true_rows, false_rows, gini(training_data))

0.07820069204152191

### Find the best question to ask 
by iterating over every feature / value and calculating the information gain.

In [62]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features): 

        values = set([row[col] for row in rows])

        for val in values:

            question = Question(col, val)

            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [63]:
best_gain, best_question = find_best_split(training_data)
best_question

Is price in dolar >= 2?

### Leaf node classifies data

Number of times it appears in the rows from the training data that reach this leaf.

In [64]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
A Decision Node asks a question. This holds a reference to the question, and to the two child nodes.


In [71]:
class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [72]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [73]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [74]:
my_tree = build_tree(training_data)

In [75]:
print_tree(my_tree)

Is price in dolar >= 2?
--> True:
  Is price in dolar >= 4?
  --> True:
    Is color == Orange?
    --> True:
      Predict {'Orange Juice': 1, 'Orange Soda': 1}
    --> False:
      Is color == Brown?
      --> True:
        Predict {'Coffee': 1, 'Hot Chocolate': 1}
      --> False:
        Is type == Fruit?
        --> True:
          Predict {'Pineapple': 1, 'Banana': 1}
        --> False:
          Predict {'Lemonade': 1}
  --> False:
    Is price in dolar >= 3?
    --> True:
      Is color == Yellow?
      --> True:
        Predict {'Lemon': 1, 'Mango': 1}
      --> False:
        Is color == Orange?
        --> True:
          Predict {'Orange': 1}
        --> False:
          Predict {'Apple': 2}
    --> False:
      Is color == Red?
      --> True:
        Predict {'Grape Juice': 1}
      --> False:
        Predict {'Green Tea': 1}
--> False:
  Predict {'Grape': 3}


In [76]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [77]:
classify(training_data[0], my_tree)

{'Apple': 2}

In [78]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [79]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [80]:
# Evaluate
testing_data = [
    ['Fruit','Yellow', 5, 'Pineapple'],
    ['Fruit','Yellow', 3, 'Mango'],
    ['Fruit','Purple', 1, 'Grape'],
    ['Fruit','Yellow', 5, 'Banana'],
    ['Drink','Brown' ,5 ,'Coffee'],
    ['Drink','Orange',4,'Orange Juice'],
    ['Drink','Red',2,'Grape Juice'],
]

In [81]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Pineapple. Predicted: {'Pineapple': '50%', 'Banana': '50%'}
Actual: Mango. Predicted: {'Lemon': '50%', 'Mango': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Banana. Predicted: {'Pineapple': '50%', 'Banana': '50%'}
Actual: Coffee. Predicted: {'Coffee': '50%', 'Hot Chocolate': '50%'}
Actual: Orange Juice. Predicted: {'Orange Juice': '50%', 'Orange Soda': '50%'}
Actual: Grape Juice. Predicted: {'Grape Juice': '100%'}
